In [1]:
# 1.1 Importing Libraries
from __future__ import print_function
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# 1.2 Setting up Device

# Enable anomaly detection to track in-place operation errors
torch.autograd.set_detect_anomaly(True)

# Ensure that the results directory exists
os.makedirs("./results", exist_ok=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 1.3 Data Loading and Preprocessing
batchSize = 64       # Batch size
imageSize = 64       # Image size (64x64)
transform = transforms.Compose([
    transforms.Resize(imageSize),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = dset.CIFAR10(root='./data', download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 43762098.56it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
# 1.4 Model Definition
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=False),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1)

# Initialize models
netG = Generator().to(device)
netG.apply(weights_init)
netD = Discriminator().to(device)
netD.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [5]:
# 1.5 Optimizer and Loss Function
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [6]:
# 1.6 Training Loop
for epoch in range(25):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        netD.zero_grad()
        real_images, _ = data
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Create real labels with label smoothing and noise
        real_label_noise = torch.rand(batch_size, device=device) * 0.1  # Small random noise
        label_real = torch.full((batch_size,), 0.9, dtype=torch.float, device=device) - real_label_noise

        # Forward pass real batch through Discriminator
        output_real = netD(real_images)
        errD_real = criterion(output_real, label_real)

        # Generate fake images from the Generator
        noise = torch.randn(batch_size, 100, 1, 1, device=device)  # nz = 100
        fake_images = netG(noise)

        # Create fake labels with noise
        fake_label_noise = torch.rand(batch_size, device=device) * 0.1
        label_fake = torch.full((batch_size,), 0.0, dtype=torch.float, device=device) + fake_label_noise

        # Forward pass fake batch through Discriminator
        output_fake = netD(fake_images.detach())
        errD_fake = criterion(output_fake, label_fake)

        # Backpropagation and optimization for Discriminator
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()

        # Generator wants to fool the discriminator, so labels are real
        output = netD(fake_images)
        errG = criterion(output, label_real)  # Use the same real labels for generator

        # Backpropagation and optimization for Generator
        errG.backward()
        optimizerG.step()

        # Print progress
        if i % 100 == 0:
            print(f'[{epoch}/{25}][{i}/{len(dataloader)}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}')

        # Save results every 100 steps
        if i % 100 == 0:
            vutils.save_image(real_images, f"./results/real_samples.png", normalize=True)
            vutils.save_image(fake_images.detach(), f"./results/fake_samples_epoch_{epoch:03d}.png", normalize=True)

[0/25][0/782] Loss_D: 1.6311 Loss_G: 2.7602
[0/25][100/782] Loss_D: 1.2368 Loss_G: 14.0166
[0/25][200/782] Loss_D: 0.9884 Loss_G: 1.2956
[0/25][300/782] Loss_D: 1.0515 Loss_G: 2.4273
[0/25][400/782] Loss_D: 0.7718 Loss_G: 2.9010
[0/25][500/782] Loss_D: 1.2904 Loss_G: 0.8423
[0/25][600/782] Loss_D: 1.0965 Loss_G: 2.2169
[0/25][700/782] Loss_D: 1.0003 Loss_G: 2.5128
[1/25][0/782] Loss_D: 1.0156 Loss_G: 2.9717
[1/25][100/782] Loss_D: 0.8505 Loss_G: 2.3725
[1/25][200/782] Loss_D: 0.8132 Loss_G: 2.1519
[1/25][300/782] Loss_D: 0.8573 Loss_G: 2.1648
[1/25][400/782] Loss_D: 0.9694 Loss_G: 2.9942
[1/25][500/782] Loss_D: 1.0387 Loss_G: 1.9655
[1/25][600/782] Loss_D: 1.1342 Loss_G: 1.4300
[1/25][700/782] Loss_D: 0.9417 Loss_G: 1.9331
[2/25][0/782] Loss_D: 1.0289 Loss_G: 2.7229
[2/25][100/782] Loss_D: 1.1840 Loss_G: 1.0380
[2/25][200/782] Loss_D: 1.1741 Loss_G: 1.4559
[2/25][300/782] Loss_D: 0.8427 Loss_G: 2.4825
[2/25][400/782] Loss_D: 0.8758 Loss_G: 2.4109
[2/25][500/782] Loss_D: 1.0467 Loss_G: 

In [7]:
# 1.7 Save and Evaluate Results
torch.save(netG.state_dict(), "./results/netG.pth")
torch.save(netD.state_dict(), "./results/netD.pth")